In [1]:
# LIBRERÍAS
import os
import pandas as pd
from shutil import copyfile
from tqdm import tqdm

## Definición de rutas y estructura

In [2]:
ruta_train_labels = "../data/train_labels.csv"
ruta_val_labels = "../data/val_labels.csv"
ruta_train_imagenes = "../data/train_set"
ruta_val_imagenes = "../data/val_set"
ruta_class_list = "../data/class_list.txt"
ruta_test_imagenes = "../data/test_set"

# CREAMOS LA ESTRUCTURA EN LA CARPETA DATA
os.makedirs("../data/images/train", exist_ok=True)
os.makedirs("../data/images/val", exist_ok=True)
os.makedirs("../data/labels/train", exist_ok=True)
os.makedirs("../data/labels/val", exist_ok=True)
os.makedirs("../data/images/test", exist_ok=True)
os.makedirs("../data/labels/test", exist_ok=True)

# CONFIGURAMOS EL TAMAÑO DE LOS BOUNDING BOXES
ancho_normalizado = 0.8  
alto_normalizado = 0.8   

In [3]:
train_labels = pd.read_csv(ruta_train_labels)
val_labels = pd.read_csv(ruta_val_labels)

## Procesamiento de datos en formato YOLO

In [ ]:
from shutil import copyfile
import os
from tqdm import tqdm

# Verificar si las carpetas o el archivo ya existen
if not os.path.exists("../data/images") or not os.path.exists("../data/labels") or not os.path.exists("../data/data.yaml"):
    print("Iniciando procesamiento de imágenes y etiquetas...")

    # Crear la estructura de carpetas si no existe
    os.makedirs("../data/images/train", exist_ok=True)
    os.makedirs("../data/images/val", exist_ok=True)
    os.makedirs("../data/images/test", exist_ok=True)
    os.makedirs("../data/labels/train", exist_ok=True)
    os.makedirs("../data/labels/val", exist_ok=True)

    # FUNCIÓN PARA PROCESAR IMÁGENES Y GENERAR BOUNDING BOXES
    def procesar_imagenes_y_labels(labels_df, ruta_imagenes, salida_imagenes, salida_labels):
        for _, row in tqdm(labels_df.iterrows(), total=labels_df.shape[0], desc=f"Procesando {salida_imagenes}"):
            # Copiar la imagen a la carpeta YOLO
            imagen_origen = os.path.join(ruta_imagenes, row['img_name'])
            imagen_destino = os.path.join(salida_imagenes, row['img_name'])
            copyfile(imagen_origen, imagen_destino)

            # Generar anotación YOLO
            etiqueta = row['label']
            x_centro, y_centro = 0.5, 0.5  # Centro de la imagen
            bounding_box = f"{etiqueta} {x_centro} {y_centro} {ancho_normalizado} {alto_normalizado}\n"

            # Guardar el archivo .txt
            ruta_etiqueta = os.path.join(salida_labels, row['img_name'].replace('.jpg', '.txt'))
            with open(ruta_etiqueta, "w") as f:
                f.write(bounding_box)

    # PROCESAMOS EL CONJUNTO DE ENTRENAMIENTO
    procesar_imagenes_y_labels(
        train_labels,
        ruta_train_imagenes,
        "../data/images/train", 
        "../data/labels/train"
    )

    # PROCESAMOS EL CONJUNTO DE VALIDACIÓN
    procesar_imagenes_y_labels(
        val_labels,
        ruta_val_imagenes,
        "../data/images/val",   
        "../data/labels/val"
    )

    # PROCESAR CONJUNTO DE TEST SIN LABELS
    def procesar_imagenes_test(ruta_imagenes, salida_imagenes):
        for imagen in tqdm(os.listdir(ruta_imagenes), desc="Procesando imágenes de prueba"):
            imagen_origen = os.path.join(ruta_imagenes, imagen)
            imagen_destino = os.path.join(salida_imagenes, imagen)
            copyfile(imagen_origen, imagen_destino)

    # PROCESAR IMÁGENES DE TEST
    procesar_imagenes_test(
        "../data/test_set",     
        "../data/images/test"
    )

    print("Procesamiento completado.")
else:
    print("Las carpetas 'images', 'labels' o el archivo 'data.yaml' ya existen. No se procesará nuevamente.")

Iniciando procesamiento de imágenes y etiquetas...


Procesando ../data/images/train:   9%|███▋                                     | 10649/118475 [01:09<11:15, 159.53it/s]

## Procesamiento en conjunto de prueba

In [ ]:
# Función para encontrar el directorio raíz del proyecto
def find_project_path(start_dir=None, project_name="tf-machine-learning"):
    if start_dir is None:
        start_dir = os.getcwd()  # Usar el directorio de trabajo actual como punto de partida
    for root, dirs, files in os.walk(start_dir, onerror=lambda e: None):
        try:
            if project_name in dirs:
                return os.path.join(root, project_name)
        except PermissionError:
            print(f"Permiso denegado en: {root}")
            continue
    raise FileNotFoundError(f"No se encontró el proyecto '{project_name}' comenzando en '{start_dir}'.")

# Detectar el directorio raíz del proyecto
project_root = find_project_path(start_dir="/Users", project_name="tf-machine-learning")
print("Directorio del proyecto configurado:", project_root)

## Creación de data.yaml

In [ ]:
# CREAR RUTAS ABSOLUTAS
train_path = os.path.join(project_root, "data/images/train")
val_path = os.path.join(project_root, "data/images/val")
yaml_path = os.path.join(project_root, "data/data.yaml")
ruta_class_list = os.path.join(project_root, "data/class_list.txt")

# Leer las clases desde el archivo class_list.txt
with open(ruta_class_list, "r") as f:
    clases = [line.strip().split(" ", 1)[1] for line in f.readlines()]

# Crear archivo data.yaml
with open(yaml_path, "w") as f:
    f.write(f"train: {train_path}\n")
    f.write(f"val: {val_path}\n")
    f.write(f"nc: {len(clases)}\n")
    f.write(f"names: {clases}\n")

print(f"Archivo data.yaml generado en: {yaml_path}")

## Limpieza de datos

In [ ]:
import shutil

# ELIMINAMOS CARPETAS Y ARCHIVOS INNECESARIOS
carpetas_a_eliminar = ["../data/train_set", "../data/val_set", "../data/test_set"]
archivos_a_eliminar = ["../data/train_labels.csv", "../data/val_labels.csv", "../data/test_set.zip", 
                       "../data/train_set.zip", "../data/val_set.zip", 
                       "../data/sample_submission.csv", "../data/ifood2019_sample_submission.csv"]


print("Inicio del proceso de eliminación de carpetas y archivos innecesarios...")

# Carpetas
for carpeta in carpetas_a_eliminar:
    if os.path.exists(carpeta):
        shutil.rmtree(carpeta)
        print(f"Carpeta eliminada: {carpeta}")
    else:
        print(f"Carpeta no encontrada (ya eliminada o no existe): {carpeta}")

# Archivos
for archivo in archivos_a_eliminar:
    if os.path.exists(archivo):
        os.remove(archivo)
        print(f"Archivo eliminado: {archivo}")
    else:
        print(f"Archivo no encontrado (ya eliminado o no existe): {archivo}")

print("Proceso de eliminación finalizado.")